In [1]:
%load_ext rpy2.ipython

: 

In [ ]:
%%R
install.packages(c("data.table", "ggplot2", "lubridate"))

library(data.table)
library(ggplot2)
library(lubridate)

(as ‘lib’ is unspecified)

















	‘/tmp/RtmpG0D9jL/downloaded_packages’


Attaching package: ‘lubridate’



    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year



    date, intersect, setdiff, union




In [4]:
%%R

# Load necessary libraries
library(data.table)
library(ggplot2)
library(lubridate)

# Read and preprocess data
data <- fread("sam_data.csv", col.names = c("date", "type", "URL", "color"))
data[, date := mdy(date)]
data[, height := runif(nrow(data), min = .25, max = 1.5)]
data[, height := height * ifelse(color == "US", -1, 1)]

UsageError: Cell magic `%%R` not found.


In [3]:
%%R

# Define color outcomes
data[color == "US" & type == "Telefon", color_out := "Phone call with POTUS"]
data[color == "US" & type == "Birebir", color_out := "One-on-one meeting with POTUS"]
data[color == "Russia" & type == "Telefon", color_out := "Phone call with Putin"]
data[color == "Russia" & type == "Birebir", color_out := "One-on-one meeting with Putin"]
data[, color_out_factor := factor(color_out, levels = sort(unique(color_out))[rev(c(4, 2, 3, 1))])]

# Update colors and shapes for black and white differentiation
colors_out <- c("black", "white", "black", "white")
shapes_out <- c(21, 21)

# Create the plot
gg1 <- ggplot(data[!is.na(color) & color != "" & !is.na(type) & !is.na(color_out)]) +
  geom_point(aes(x = date, y = height, shape = color_out_factor, fill = color_out_factor), size = 3, position = position_dodge(width = 0.1)) +
  geom_segment(aes(x = date, xend = date, y = 0, yend = height - 0.05 * sign(height)), color = "black") +
  coord_flip() +
  theme_minimal() + # Use a minimal theme to remove the box
  theme(panel.grid.major = element_line(color = "grey80"), # Lightly shaded lines
        panel.grid.minor = element_blank(), 
        legend.position = "right") +
  scale_fill_manual(values = colors_out) +
  scale_shape_manual(values = shapes_out) +
  scale_x_date(limits = c(as.Date("2011-01-01"), as.Date("2023-12-30")),
               labels = 2011:2023,
               breaks = seq.Date(as.Date("2011-01-01"), as.Date("2023-12-30"), "year")) +
  labs(x = "", y = "", fill = "Event Type") +
  scale_y_continuous(labels = NULL) +
  geom_segment(aes(x = as.Date("2011-01-01"), xend = as.Date("2023-12-30"), y = 0, yend = 0), color = "black") +
  guides(fill = guide_legend(override.aes = list(shape = 21, fill = c("black", "white"))), shape = "none") +
  annotate("text", x = as.Date("2023-12-30"), y = 0.5, label = "Putin", hjust = 0, vjust = 0, size = 5, fontface = "bold") +
  annotate("text", x = as.Date("2023-12-30"), y = -0.5, label = "POTUS", hjust = 1, vjust = 0, size = 5, fontface = "bold")

UsageError: Cell magic `%%R` not found.


In [ ]:
%%R

pdf("table.pdf", height = 8, width = 6)
print(gg1)
dev.off()


png 
  2 
